In [1]:
import xarray as xr
from utils.data import *
from utils.utils import *
import matplotlib.pyplot as plt
import seaborn as sns
import proplot

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [234]:
province = 'ankara'
temp_threshold = 30
lu_year = 2015 # 2000, 2015
chirts_year = 2015 # 2000, 2005, 2010, 2015

# numbers starting with 2 or 3: urban
# numbers starting with 1 except 10: rural
# number 10: water
urban_tiles = [21,22,23,30]
rural_tiles = [11,12,13]

In [235]:
ds_tmax = xr.open_dataset(fr'data/common/chirts/chirts_tmax_{chirts_year}.nc').tmax
ds_tmax = ds_tmax.rio.write_crs(4326)

In [236]:
ds_lu = retrieve_ghs(province=province).sel(time=lu_year)

In [237]:
ds_tmax_clipped = clip_subroutine(ds_tmax, province, 'X', 'Y')

In [238]:
ds_threshold_days = xr.where(ds_tmax_clipped>=temp_threshold, 1, 0).sum(dim='T')

In [239]:
_, ds_lu_repr = regrid_match(ds_threshold_days, ds_lu, ds_threshold_days.rio.crs, ds_lu.rio.crs)

In [240]:
ds_lu_repr = xr.where(ds_lu_repr<0, np.nan, ds_lu_repr)

In [241]:
ds_lu_repr_class = classify_urban_rural(ds_lu_repr, urban_tiles, rural_tiles) 

In [242]:
# interpolate
Y_dir, X_dir = ds_lu_repr_class.indexes.values()
ds_threshold = ds_threshold_days.interp(X=X_dir, Y=Y_dir, method="nearest") # interp to model grid

In [243]:
urban_threshold_days = xr.where(ds_lu_repr_class==1, ds_threshold, np.nan)
rural_threshold_days = xr.where(ds_lu_repr_class==0, ds_threshold, np.nan)

In [244]:
print(np.nanmedian(urban_threshold_days))
print(np.nanmedian(rural_threshold_days))

49.0
49.0


In [185]:
print(np.max(urban_threshold_days).values)
print(np.max(rural_threshold_days).values)

42.0
44.0
